# Esercitazione su k-Means e Hierarchical Clustering

## Indice contenuti
- [Obiettivo esercitazione](#Obiettivo-esercitazione)
- [Descrizione ed analisi del dataset](#Descrizione-ed-analisi-del-dataset)
- [Analisi esplorativa del dataset](#Analisi-esplorativa-del-dataset)
    - [Caricamento in memoria del dataset](#Caricamento-in-memoria-del-dataset)
    - [Pulizia del dataset](#Pulizia-del-dataset)
    - [Data Preparation](#Data-Preparation)
    - [Trattamento Outliers](#Trattamento-Outliers)
- [Clustering via k-Means](#Clustering-via-k-Means)
    - [k-Means++](#kMeans++)
    - [Trovare il numero ottimale di clusters](#Trovare-il-numero-ottimale-di-clusters)
        - [Metodo Elbow](#Metodo-Elbow)
        - [Analisi di Silhouette](#Analisi-di-Silhouette)
    - [BoxPlot ottenuti con k-Means](BoxPlot-ottenuti-con-k-Means)
- [Clustering gerarchico](#Clustering-gerarchico)
    - [Agglomerative clustering](#Agglomerative-Clustering)
- [DBSCAN](#DBSCAN)
- [Analisi dei risultati ottenuti](#Analisi-dei-risultati-ottenuti)
<hr>

## Obiettivo esercitazione
L'esercitazione ha l'obiettivo di applicare su un dataset reale i differenti algoritmi di clustering, in particolare k-Means e Hierarchical Clustering.

Si effettueranno, inoltre, differenti variazioni all'applicazione standard degli algoritmi per comprendere l'utilizzo dei differenti iper-parametri a seconda delle documentazioni ufficiali dei metodi utilizzati.

## Descrizione ed analisi del dataset
Il dataset che verrà utilizzato è disponibile su <a href="https://archive.ics.uci.edu/ml/datasets/online+retail">UCI</a> e tratta informazioni circa le transazioni di acquisti online effettuati tra il 01/12/2010 e il giorno 09/12/2011 al fine di identificare dei cluster tra la clientela in base agli acquisti effettuati.

In particolare, si desidera applicare la segmentazione degli utenti in base ad un fattore RFM che tiene conto dei seguenti aspetti:

- <b>R (Recency)</b>: Numero di giorni trascorsi dall'ultimo acquisto
- <b>F (Frequency)</b>: Numero di transazioni effettuate
- <b>M (Monetary)</b>: Ammontare economico delle transazioni registrate

Le features presenti nel dataset sono le seguenti:
- <b>InvoiceNo</b>: Numero della fattura. Tipo di dato nominale, univoco e espresso su 6 cifre. _Se il codice comincia per la lettera 'c', indica la cancellazione dell'ordine.
- <b>StockCode</b>: Codice univoco del prodotto acquistato, espresso su 5 cifre.
- <b>Description</b>: Descrizione del prodotto. Tipo di dato nominale.
- <b>Quantity</b>: Quantità di ciascun prodotto acquistato in una singola transazione. Tipo di dato numerico.
- <b>InvoiceDate</b>: Timestamp dell'emissione della fattura. Tipo di dato numerico.
- <b>UnitPrice</b>: Prezzo unitario del prodotto acquistato in sterline. Tipo di dato numerico.
- <b>CustomerID</b>: Codice univoco dell'acquirente. Tipo di dato nominale.
- <b>Country</b>: Nome della nazione di residenza dell'acquirente. Tipo di dato nominale.

## Analisi esplorativa del dataset

In [ ]:
# Import delle l'analisi esplorativa dei dati
import numpy as np
import pandas as pd
import seaborn as sns
import datetime as dt
import matplotlib.pyplot as plt

# import delle librerie richieste per l'applicazione di algoritmi di clustering
import sklearn
from sklearn.cluster import KMeans
from scipy.cluster.hierarchy import linkage
from sklearn.metrics import silhouette_score
from scipy.cluster.hierarchy import cut_tree
from scipy.cluster.hierarchy import dendrogram
from sklearn.preprocessing import StandardScaler

### Caricamento in memoria del dataset

Con il seguente comando si effettua il caricamento in memoria di quanto contenuto nel dataset _'OnlineRetail.csv'_.

Per condurre una prima fase di analisi esplorativa e comprendere la natura dei dati a disposizione, si stampano di seguito i primi cinque esempi presenti nel dataset:

In [ ]:
#encoding: Encoding to use for UTF when reading/writing
#header
store = pd.read_csv('../input/online-retail-ii-uci/online_retail_II.csv', sep=",", encoding="ISO-8859-1", header=0)
store.head()

Per ottenere informazioni statistiche inerenti ciascuna feature a disposizione, mediante il metodo _describe()_ si è provveduto al calcolo delle seguenti informazioni:
- <b>count</b>: conteggio del numero di esempi per la feature selezionata
- <b>mean</b>: media aritmetica per la feature selezionata
- <b>std</b>: deviazione standard per la feature selezionata
- <b>min</b>: valore minimo presentato dagli esempi per la feature selezionata
- <b>25%</b>: primo quartile calcolato sugli esempi per la feature selezionata
- <b>50%</b>: secondo quartile calcolato sugli esempi per la feature selezionata
- <b>75%</b>: terzo quartile calcolato sugli esempi per la feature selezionata
- <b>max</b>: valore massimo presentato dagli esempi per la feature selezionata

In [ ]:
store.describe()

Successivamente, al fine di comprendere le dimensioni (in termini di esempi e di features a disposizione), mediante apposito attributo si stampano il numero di righe e di colonne del DataFrame:

In [ ]:
store.shape

Al fine di ottenere una descrizione complessiva del Dataframe (e dunque del relativo dataset) caricato, mediante il metodo _info()_ si sono ottenute le seguenti informazioni:
- <b>#</b>: numero di feature presente nel DataFrame
- <b>Column</b>: intestazione delle features nel DataFrame
- <b>Non-Null Count</b>: contatore di valori non nulli per ogni feature presente nel DataFrame
- <b>Dtype</b>: tipo di dato memorizzato per ogni feature presente nel DataFrame

In [ ]:
store.info()

In [ ]:
store.describe()

## Pulizia del dataset

### Gestione dei valori mancanti
Al fine di gestire propriamente i dati mancanti, di seguito è realizzata una funzione che indichi, per ogni feature presente nel dataset, la percentuale dei dati mancanti.

Come è possibile osservare, le seguenti features presentano valori nulli:
- Description
- CustomerID

In [ ]:
df_null = round(100*(store.isnull().sum())/len(store), 2)
df_null

Dato l'alto numero di esempi presenti nel dataset, si decide di rimuovere le istanze che presentino valori nulli. Tale operazione è svolta utilizzando il metodo _dropna()_. 

Successivamente, invece, si provvede a ristampare il nuovo numero di esempi e di features che presenta il dataset.

In [ ]:
store = store.dropna()
store.shape

## Data Preparation
Secondo quanto anticipato in apertura, si provvede ad introdurre nuove features per poter valutare il comportamento dei clienti. In particolare, si definiscono le seguenti tre features:
- <b>R (Recency)</b>: Numero di giorni trascorsi dall'ultimo acquisto
- <b>F (Frequency)</b>: Numero di transazioni effettuate
- <b>M (Monetary)</b>: Ammontare economico delle transazioni registrate

In [ ]:
# Per effettuare operazioni di Join, il tipo di dato CustomerID viene convertito in tipo String
store['Customer ID'] = store['Customer ID'].astype(str)

In [ ]:
#Introduzione del nuovo attributo Monetary
store['Amount'] = store['Quantity']*store['Price']
rfm_m = store.groupby('Customer ID')['Amount'].sum()
rfm_m = rfm_m.reset_index()
rfm_m.head()

In [ ]:
# Introduzione del nuovo attributo Frequency

rfm_f = store.groupby('Customer ID')['Invoice'].count()
rfm_f = rfm_f.reset_index()
rfm_f.columns = ['Customer ID', 'Frequency']
rfm_f.head()

In [ ]:
# Unione dei due dataframe: corrisponde ad un Inner-JOIN SQL

rfm = pd.merge(rfm_m, rfm_f, on='Customer ID', how='inner')
rfm.head()

In [ ]:
# Conversione della data nel tipo supportato da Python DateTime per effettuare le dovute operazioni
store['InvoiceDate'] = pd.to_datetime(store['InvoiceDate'],format='%Y-%m-%d %H:%M')

In [ ]:
# Calcolo della data massima registrata all'interno del dataset
max_date = max(store['InvoiceDate'])
max_date

In [ ]:
# Calcolo della differenza tra la data massima registrata nel dataset e il valore espresso per _InvoiceDate_
store['Diff'] = max_date - store['InvoiceDate']
store.head()

In [ ]:
# Raggruppando gli esempi per CustomerID, si prende il valore minore della data
rfm_p = store.groupby('Customer ID')['Diff'].min()
rfm_p = rfm_p.reset_index()
rfm_p.head()

In [ ]:
# Introduzione della feature Recency, estrapolando dalla data solo il numero di giorni
rfm_p['Diff'] = rfm_p['Diff'].dt.days
rfm_p.head()

In [ ]:
# Unione dei dataframe, al fine di ottenere l'ultimo DataFrame complessivo
rfm = pd.merge(rfm, rfm_p, on='Customer ID', how='inner')
#Intestazione delle colonne
rfm.columns = ['Customer ID', 'Amount', 'Frequency', 'Recency']
#Stampa dei primi 5 esempi
rfm.head()

## Trattamento Outliers
Per rimuovere gli outliers presenti nel dataset, inizialmente si realizza un boxplot nel quale vengono plottati gli esempi rispetto agli attributi _Amount_, _Frequency_ e _Recency_.

Successivamente, onde evitare problemi circa la presenza di dati espressi su un diverso range numerico, si effettua la standardizzazione, mediante apposito metodo _StandardScaler()_ applicata sul dataset.

In [ ]:
attributes = ['Amount','Frequency','Recency']
plt.rcParams['figure.figsize'] = [10,8]
sns.boxplot(data = rfm[attributes], orient="v", palette="Set2" ,whis=1.5,saturation=1, width=0.7)
plt.title("Outliers nel dataset", fontsize = 14, fontweight = 'bold')
plt.ylabel("Range", fontweight = 'bold')
plt.xlabel("Attributes", fontweight = 'bold')

In [ ]:
# Rimozione degli outliers per Amount utilizzando InterQuartileRange
Q1 = rfm.Amount.quantile(0.05)
Q3 = rfm.Amount.quantile(0.95)
IQR = Q3 - Q1
rfm = rfm[(rfm.Amount >= Q1 - 1.5*IQR) & (rfm.Amount <= Q3 + 1.5*IQR)]

# Rimozione degli outliers per Recency utilizzando InterQuartileRange
Q1 = rfm.Recency.quantile(0.05)
Q3 = rfm.Recency.quantile(0.95)
IQR = Q3 - Q1
rfm = rfm[(rfm.Recency >= Q1 - 1.5*IQR) & (rfm.Recency <= Q3 + 1.5*IQR)]

# Rimozione degli outliers per Frequency utilizzando InterQuartileRange
Q1 = rfm.Frequency.quantile(0.05)
Q3 = rfm.Frequency.quantile(0.95)
IQR = Q3 - Q1
rfm = rfm[(rfm.Frequency >= Q1 - 1.5*IQR) & (rfm.Frequency <= Q3 + 1.5*IQR)]

In [ ]:
#Scaling degli attributi
rfm_df = rfm[['Amount', 'Frequency', 'Recency']]

sc = StandardScaler()
df_scaled = sc.fit_transform(rfm_df)
#Stampa delle nuove dimensioni
df_scaled.shape

In [ ]:
#Conversione a dataframe
df_scaled = pd.DataFrame(df_scaled)
#Intestazione delle colonne
df_scaled.columns = ['Amount', 'Frequency', 'Recency']
#Stampa dei primi 5 esempi standardizzati
df_scaled.head()

## Clustering via k-Means

k-Means è un algoritmo di clustering non supervisionato, tra i più semplici e popolari messi a disposizione dalla libreria _Sci-Kit_.

Un cluster è definito come un insieme di punti dati che il clustering è uno degli algoritmi di apprendimento automatico non supervisionati più semplici e popolari.

Definito il valore del parametro k, che esplica il numero di centroidi da identificare nel dataset. Un centroide è la posizione immaginaria o reale che rappresenta il centro di ciascun cluster.

L'algoritmo prevede l'assegnazione di ogni punto dati viene a ciascuno dei cluster utilizzando la nozione di distanza. In altre parole, l'algoritmo k-Means identifica il numero k di centroidi e quindi assegna ogni punto dati al cluster più vicino, mantenendo i centroidi i più piccoli possibili.

Per clusterizzare i dati presenti nel dataset, l'algoritmo k-Means identifica randomicamente un primo gruppo di centroidi e tali sono utilizzati come punti iniziali per ogni cluster. Successivamente, si effettua il ricalcolo dei centroidi ogni qualvolta un nuovo esempio è assegnato al cluster, al fine di ottimizzare le posizioni dei centroidi.
Il processo di ottimizzazione termina quando si raggiunge il numero delle iterazioni massime (definite) oppure quando si è giunti alla convergenza del metodo.

### kMeans++
Nel k-means classico, si utilizza un seme casuale per posizionare i centroidi iniziali, che a volte può provocare cattivi raggruppamenti o una lenta convergenza se i centroidi iniziali sono scelti male. Un modo per risolvere questo problema è eseguire l'algoritmo k-mean più volte su un set di dati e scegliere il modello con le migliori prestazioni in termini di SSE.

Un'altra strategia è quella di posizionare i centroidi iniziali molto distanti tra loro tramite l'algoritmo k-means ++, che porta a risultati migliori e più coerenti rispetto ai classici k-mean 

Per utilizzare il k-Means++ basterà porre l'attributo init = 'k-means++' (che è già posto di default). Per utilizzare il k-Means classico bisognerà porre l'attributo init = 'random'.

In [ ]:
# Metodo k-Means con un numero di clusters arbitrario.
# - n_clusters: numero di cluster desiderati (3) - limitazione del k-Means;
# - n_init: esegue l'algoritmo n volte in modo indipendente, con diversi centroidi casuali per scegliere il modello finale come quello con il SSE più basso.
# - max_iter: indica il numero massimo di iterazioni per ogni singola esecuzione (qui, 300). 


#method = KMeans(n_clusters=4, random_state = 1, max_iter=300, tol=1e-04, init='random', n_init=10)
method = KMeans(n_clusters=4, random_state = 1, max_iter=300, tol=1e-04, init='k-means++', n_init=10)
method.fit(df_scaled)

In [ ]:
#Stampa delle etichette relative ai cluster
method.labels_

### Trovare il numero ottimale di clusters

#### Metodo Elbow
Al fine di identificare il giusto numero per il parametro _n_clusters_ è possibile definire un metodo grafico che consenta, variando il parametro mediante una lista di valori espressi, di poter valutare l'attributo _intertia_ (ovvero la somma della radice delle distanze dei campioni dal centro del cluster più vicino).

In [ ]:
elbow_values = []
range_n_clusters = [1, 2, 3, 4, 5, 6, 7, 8]
for num_clusters in range_n_clusters:
    method = KMeans(n_clusters=num_clusters, random_state = 1, max_iter=300, tol=1e-04, init='k-means++', n_init=10)
    method.fit(df_scaled)
    
    elbow_values.append(method.inertia_)
    
# Plot del valore della somma della radice delle distanze al crescere del numero dei cluster
plt.plot(range(1, 9), elbow_values, marker='o')
plt.ylabel("Sum of Squared Distance")
plt.xlabel("Numero dei cluster")

#### Analisi di Silhouette

L'analisi di Silhouette si riferisce a un metodo di interpretazione e convalida della coerenza dei dati rispetto ai cluster identificati.

Il valore dek coefficiente di Silhouette è una misura di quanto un oggetto sia simile al proprio cluster (coesione) rispetto ad altri cluster (separazione). Tale valore è espresso in un intervallo [-1, +1], dove un valore alto indica che l'esempio è ben adattato al proprio cluster e scarsamente abbinato ai cluster vicini. Se la maggior parte degli oggetti ha un valore elevato, la suddivisione degli esempi nei rispettivi cluster è appropriata. Se molti punti, invece, hanno un valore basso o negativo, la suddivisione degli esempi nei cluster definiti potrebbe risultare inappropriata.

L'analisi di Silhouette può essere condotta utilizzando una qualsiasi metrica di distanza, come la distanza euclidea o la distanza di Manhattan.
In particolare, può essere espressa come segue:
$$\text{silhouette score}=\frac{p-q}{max(p,q)}$$

$p$ è la distanza media tra il punto e il centroide del cluster più vicino.

$q$ è la distanza media intra-cluster definita su tutti i punti presenti nel proprio cluster.


In [ ]:
# Definizione della lista del numero di cluster da testare
range_n_clusters = list(x for x in range (2,10+1))

for num_clusters in range_n_clusters:
    method = KMeans(n_clusters=num_clusters, max_iter=50)
    method.fit(df_scaled)
    cluster_labels = method.labels_
    # Calcolo coefficiente di silhouette
    silhouette_avg = silhouette_score(df_scaled, cluster_labels)
    print("Per n_clusters={0}, il coefficiente di Silhouette è pari a {1}".format(num_clusters, silhouette_avg))

Ottenuti i risultati del coefficiente di Silhouette, si sceglie il valore 3 per il parametro _n_clusters_ e si riesegue nuovamente l'algoritmo k-Means registrando le relative etichette.

In [ ]:
method = KMeans(n_clusters=3, random_state = 1, max_iter=300, tol=1e-04, init='k-means++', n_init=10)
method.fit(df_scaled)

In [ ]:
method.labels_

In [ ]:
# Assegnazione delle etichette a ciascun esempio presente nel DataFrame
rfm['Cluster_Id'] = method.labels_
# Stampa dei primi 5 esempi
rfm.head()

### BoxPlot ottenuti con k-Means

In [ ]:
features_list = ['Amount', 'Frequency', 'Recency']

for feature in features_list:
    sns.boxplot(x='Cluster_Id', y=feature, data=rfm)
    plt.show()

## Clustering gerarchico

Gli algoritmi gerarchici in genere clusterizzano i dati usando le misure di distanza. Tuttavia, l'uso delle funzioni di distanza non è obbligatorio. Molti algoritmi gerarchici utilizzano altri metodi di clustering, ad esempio metodi density-based o graph-based, come subroutine per la costruzione della gerarchia.

Uno dei motivi principali di utiulizzo di tale modalità di clustering è che diversi livelli di granularità del clustering forniscono dei dettagli specifici per l'applicazione. Ciò fornisce una tassonomia di cluster, che possono essere esplorati sulla base di tali dettagli semantici.

L'organizzazione gerarchica consente la navigazione manuale molto conveniente per un utente, specialmente quando il contenuto dei cluster può essere descritto in modo semanticamente comprensibile. In altri casi, tali organizzazioni gerarchiche possono essere utilizzate dagli algoritmi di indicizzazione, rispetto alle macroaree di riferimento.
Inoltre, tali metodi possono talvolta essere utilizzati anche per creare cluster "piatti" migliori (dove tutte le categorie sono posto allo stesso livello). Alcuni metodi gerarchici agglomerativi e metodi di divisione, possono fornire cluster di qualità migliore rispetto ai metodi di partizionamento come k-Means, sebbene con un costo computazionale più elevato.

Esistono due tipi di algoritmi gerarchici, a seconda di come viene costruito l'albero gerarchico dei cluster:
- Metodi bottom-up (agglomerativi): i singoli punti dati vengono successivamente agglomerati in cluster di livello superiore. La principale variazione tra i diversi metodi è nella scelta della funzione obiettivo utilizzata per fondere i cluster.
- Metodi top-down (divisivi): un approccio top-down viene utilizzato per partizionare successivamente i punti in una struttura ad albero. Un algoritmo di clustering piatto può essere utilizzato per il partizionamento in un determinato passo. Tale approccio offre un'enorme flessibilità in termini di scelta del compromesso tra l'equilibrio nella struttura ad albero e l'equilibrio nel numero di punti in ciascun nodo della struttura ad albero.

**Single Linkage:<br>**

Nel clustering che sfrutta la modalità di collegamento _single linkage_, la distanza tra due cluster è definita come la più piccola distanza calcolabile tra due punti in ciascun cluster. Per esempio, la distanza tra il cluster “r” e “s” è uguale alla lunghezza dell'arco tra i due punti più vicini, così come visibile dalla figura riportata.

![](https://www.saedsayad.com/images/Clustering_single.png)

In [ ]:
# Applicazione del metodo Single Linkage
single_linkage = linkage(df_scaled, method="single", metric='euclidean')
dendrogram(single_linkage)
plt.show()

**Complete Linkage<br>**

Nel metodo _Complete Linkage_, la distanza tra due cluster è definita come la più grande distanza tra due punti in ciascun cluster.

Per esempio, la distanza tra i cluster “r” e “s” è uguale alla lunghezza dell'arco tra i due punti più distanti dei due cluster.

![](https://www.saedsayad.com/images/Clustering_complete.png)

In [ ]:
# Applicazione del metodo Complete linkage
complete_linkage = linkage(df_scaled, method="complete", metric='euclidean')
dendrogram(complete_linkage)
plt.show()

**Average Linkage:<br>**

Con il metodo _Average Linkage_, la distanza tra due cluster è definita come la distanza media presente tra ciascun punto di un cluster con tutti i punti dell'altro cluster.

Per esempio, la distanza tra i cluster “r” e “s” è uguale alla lunghezza mediata dell'arco che connette i punti di un cluster all'altro.

![](https://www.saedsayad.com/images/Clustering_average.png)

In [ ]:
# Applicazione del metodo Average linkage
avg_linkage = linkage(df_scaled, method="average", metric='euclidean')
dendrogram(avg_linkage)
plt.show()

#### Taglio del Dendrogramma in base al valore di K

In [ ]:
# Desiderando un numero di cluster pari a 3, si inizializza il parametro n_clusters=3
cluster_labels = cut_tree(avg_linkage, n_clusters=3).reshape(-1, )
#Stampa delle etichette dei cluster
cluster_labels

In [ ]:
# Assegnazione delle etichette a ciascun esempio presente nel DataFrame
rfm['Cluster_Labels'] = cluster_labels
# Stampa dei primi 5 elementi presenti nel DataFrame
rfm.head()

In [ ]:
#Plot delle Features

features_list = ['Amount', 'Frequency', 'Recency']

for feature in features_list:
    sns.boxplot(x='Cluster_Labels', y=feature, data=rfm)
    plt.show()

### Agglomerative Clustering

Nel clustering agglomerativo, come già accennato, i singoli punti dati vengono agglomerati iterativamente in cluster di livello superiore.
Nel primo step, ogni singolo punto costituisce un cluster. Successivamente, si agglomerano insieme via via sempre più punti, andando a costruire cluster sempre più popolati.
Il metodo si arresta quando si raggiunge un certo numero di cluster.
Nel metodo seguente vengono utilizzati i seguenti parametri:
- <b>n_clusters=3</b>: si desiderano tre cluster come suggerito dal metodo Elbow
- <b>affinity</b>: metrica utilizzata per computare il linkage. Si utilizza la distanza euclidea.
    - <b>euclidean</b>
    - <b>l1</b>
    - <b>l2</b>
    - <b>manhattan</b>
    - <b>cosine</b>
    - <b>precomputed</b>
- <b>linkage</b>: criterio di collegamento da utilizzare. Ne esistono diversi:
    - <b>ward</b>: minimizza la varianza dei cluster che devono essere fusi insieme
    - <b>average</b>: usa la media delle distanze di ogni osservazione nei due insiemi
    - <b>complete</b>: usa la distanza massima tra due punti negli insiemi
    - <b>single</b>: usa la distanza minima tra due insiemi

In [ ]:
from sklearn.cluster import AgglomerativeClustering

ac = AgglomerativeClustering(n_clusters=3, affinity='euclidean', linkage='complete')
agglomerative_cluster_labels = ac.fit_predict(df_scaled)

In [ ]:
# Assegnazione delle etichette a ciascun esempio presente nel DataFrame
rfm['Agglomerative_Clustering'] = agglomerative_cluster_labels
# Stampa dei primi 5 elementi presenti nel DataFrame
rfm.head()

In [ ]:
#Plot delle Features

features_list = ['Amount', 'Frequency', 'Recency']

for feature in features_list:
    sns.boxplot(x='Agglomerative_Clustering', y=feature, data=rfm)
    plt.show()

### DBSCAN

DBSCAN è un algoritmo di clustering Density-Based utilizzabile su dataset che presentano punti rumorosi. È un algoritmo non parametrico di clustering basato sulla densità: dato un insieme di punti in uno spazio, raggruppa i punti che sono altamente vicini, contrassegnando come punti anomali i punti che si trovano da soli in regioni a bassa densità. 

DBSCAN è uno degli algoritmi di clustering più comuni e anche i più citati nella letteratura scientifica e presenta i seguenti vantaggi:
- Non richiede la specifica a priori di un numero di cluster, a differenza di k-Means
- Gestione accurata dei punti rumorosi
- Robusto in presenza degli outliers

Per DBSCAN, invece, si identificano i seguenti svantaggi:
- Non deterministico: i punti presenti sulle frontiere possono essere assegnati a cluster differenti, in base all'ordine in cui i dati sono processati
- La qualità dei risultati restituiti da DBSCAN dipende dalla misura di distanza usata
- Sensibile al fenomeno della "Curse of dimensionality" in presenza di dataset con un numero di features elevato

In [ ]:
from sklearn.cluster import DBSCAN
dbscan = DBSCAN(metric='euclidean')
dbscan.fit(df_scaled)

In [ ]:
dbscan_labels = dbscan.labels_
#Stampa delle etichette dei cluster
dbscan_labels

In [ ]:
# Assegnazione delle etichette a ciascun esempio presente nel DataFrame
rfm['DensityBased_Labels'] = dbscan_labels
# Stampa dei primi 5 elementi presenti nel DataFrame
rfm.head()

In [ ]:
#Plot delle Features

features_list = ['Amount', 'Frequency', 'Recency']

for feature in features_list:
    sns.boxplot(x='DensityBased_Labels', y=feature, data=rfm)
    plt.show()

## Analisi dei risultati ottenuti

Di seguito vengono riportate le interpretazioni dei risultati ottenuti dopo l'applicazione e i relativi cluster restituiti da ciascun algoritmo mostrato.

<hr>

**k-Means**, dopo aver definito un numero di cluster pari a 3, ha portato all'identificazione dei seguenti risultati:
- I clienti con ClusterID=1 sono quei clienti che hanno un alto numero di transizioni se posti a confronto con gli altri clienti
- I clienti con ClusterID=1 sono clienti che acquistano più frequentemente rispetto agli altri clienti
- I clienti con ClusterID=2 sono clienti che non hanno acquistato recentemente e, pertanto, destano poco interesse dal punto di vista di business

<hr>

**Clustering Gerarchico**, dopo aver definito un numero di cluster pari a 3, ha portato all'identificazione dei seguenti risultati:
- I clienti con Cluster_Labels=2 sono quei clienti che hanno un alto numero di transizioni se posti a confronto con gli altri clienti
- I clienti con Cluster_Labels=2 sono clienti che acquistano più frequentemente rispetto agli altri clienti
- I clienti con Cluster_Labels=0 sono clienti che non hanno acquistato recentemente e, pertanto, destano poco interesse dal punto di vista di business

<hr>

**DBSCAN** ha restituito ben 7 cluster che identificano la naturale suddivisione dei dati rispetto ai dati a disposizione, potendo vedere dai precedenti boxplot, le differenti distribuzioni per le tre features oggetto di studio.